In [1]:
print('hello')

hello


In [2]:
import pandas as pd

# Creating a dictionary with some sample data
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David'],
    'Age': [24, 27, 22, 32],
    'City': ['New York', 'Los Angeles', 'Chicago', 'Houston']
}

# Creating a DataFrame using the dictionary
df = pd.DataFrame(data)

# Displaying the DataFrame
print(df)


      Name  Age         City
0    Alice   24     New York
1      Bob   27  Los Angeles
2  Charlie   22      Chicago
3    David   32      Houston


In [3]:
import pytesseract
from PIL import Image
import os

# Path to the directory containing your image files
folder_path = './files'

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
        # Open the image file
        img_path = os.path.join(folder_path, filename)
        img = Image.open(img_path)

        # Use Tesseract to extract text from the image
        text = pytesseract.image_to_string(img)
        
        # Print the extracted text
        print(f'Text from {filename}:\n{text}\n')


TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.

In [5]:
import easyocr
import os
import cv2

# Create an EasyOCR reader
reader = easyocr.Reader(['en'])  # Specify the languages you need, e.g., 'en' for English

# Path to the directory containing your image files
folder_path = './files'

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    print(filename)
    if filename.endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
        # Path to the image file
        img_path = os.path.join(folder_path, filename)
        print(img_path)
        # Read the image using OpenCV
        img = cv2.imread(img_path)

        if img is None:
            print(f"Error: The image file {img_path} is empty or not found.")
            continue  # Skip to the next file if the image is empty

        # Resize the image (e.g., resize to 2000 pixels on the longest side)
        max_dimension = 2000
        height, width = img.shape[:2]
        if max(height, width) > max_dimension:
            scaling_factor = max_dimension / float(max(height, width))
            new_size = (int(width * scaling_factor), int(height * scaling_factor))
            img = cv2.resize(img, new_size, interpolation=cv2.INTER_AREA)
            cv2.imwrite(img_path, img)

        # Use EasyOCR to extract text from the image
        result = reader.readtext(img_path)
        text = ' '.join([res[1] for res in result])
        
        # Print the extracted text
        print(f'Text from {filename}:\n{text}\n')


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


20241226_013233.jpg
./files\20241226_013233.jpg
Text from 20241226_013233.jpg:
HR Ponovite korake dok Vam ne ponestane Vrpce: EN Repeat step &-d untll you have Just a Ilttle thick ramalnlng; Ismetelje a2 &)-d) lepeseket & zslnor vegelg: MZ Blr U2 qalin Ip qulana qodor &-d addimlarni tokrarloyin; FU Kipetey a tabadtno esaurire la lunghezza del cordoncina DG nlobropere crbnkare Or "" Ao %r" AO MjvepnBaNe HA AbnxkMara Ha LT Pakartokite $-d IIngsnlus; kol balgsls vlrvele; opbbta LV Atkartot darbibas &-d Iidz beldzas auklas garums DIH Ponovite 4 d dok Vam ne ponestane uzice PL Powtorzyk krokl 4-da2 do momentu wyczerpanla dlugolcl Opakulte kroky al do konce delky provaU: wnurka, DE Wlederholen Sle dle Schrhte # d; blydie LAnge der Kordel erschoph IJL PT Repetir 0S Dassos &-d a1d sa esgolar 0 comprimento do cordel: Koriake samme d,kuni nobr plkkus 4440 0tya. RO Repatafil pajlla-d pJnd (Andva rJmdne doar pufin cordon Replta Ios pasos d haste egotar Ia longitud del cable As Ponovlte karake &-d 